In [2]:
from dataset import *
import vocab

In [3]:
activity_vocab = vocab.load_vocab(None, 0)

Using the saved vocab.


In [11]:
val_train = "train" # change to train or val
root = "/datasets/home/71/171/juh079/{}".format(val_train)
annotation_file = "{}_annotations.txt".format(root)

dataset = VideoFrameDataset(
    root_path=root,
    annotationfile_path=annotation_file,
    vocab=activity_vocab,
    num_segments=10,
    frames_per_segment=1,
    imagefile_template='img_{:05d}.jpg',
    transform=ImglistToTensor(),
)

In [12]:
loader = torch.utils.data.DataLoader(dataset, collate_fn=collate_fn, batch_size=2, shuffle=True)

In [13]:
for a in loader:
    print(a)
    print(a[0].shape)
    break

(tensor([[[[[0.6039, 0.5804, 0.5608,  ..., 0.2118, 0.2039, 0.2000],
           [0.5961, 0.5922, 0.5529,  ..., 0.2235, 0.2275, 0.2353],
           [0.6314, 0.4549, 0.2980,  ..., 0.2471, 0.2510, 0.2588],
           ...,
           [0.3843, 0.3843, 0.3882,  ..., 0.4000, 0.4039, 0.4039],
           [0.3922, 0.3961, 0.4000,  ..., 0.4078, 0.4078, 0.4118],
           [0.4039, 0.4039, 0.4039,  ..., 0.4000, 0.3961, 0.4039]],

          [[0.5843, 0.5804, 0.5922,  ..., 0.2118, 0.2039, 0.1961],
           [0.5686, 0.5647, 0.5451,  ..., 0.2235, 0.2275, 0.2353],
           [0.5804, 0.4000, 0.2431,  ..., 0.2471, 0.2510, 0.2588],
           ...,
           [0.6118, 0.6118, 0.6157,  ..., 0.5804, 0.5804, 0.5804],
           [0.6039, 0.6078, 0.6118,  ..., 0.5843, 0.5843, 0.5804],
           [0.6000, 0.6000, 0.6000,  ..., 0.5765, 0.5725, 0.5725]],

          [[0.5686, 0.5490, 0.5412,  ..., 0.2196, 0.2118, 0.2157],
           [0.5451, 0.5333, 0.4980,  ..., 0.2235, 0.2275, 0.2431],
           [0.5490, 0.364